<a href="https://colab.research.google.com/github/zhelampun/news_compare/blob/master/neuro_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU, LSTM
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train=pd.read_json('drive/MyDrive/train.json')  

In [ ]:
print(train)


                                                   text     id sentiment
0     Досудебное расследование по факту покупки ЕНПФ...   1945  negative
1     Медики рассказали о состоянии пострадавшего му...   1957  negative
2     Прошел почти год, как железнодорожным оператор...   1969  negative
3     По итогам 12 месяцев 2016 года на территории р...   1973  negative
4     Астана. 21 ноября. Kazakhstan Today - Агентств...   1975  negative
...                                                 ...    ...       ...
8258  Как мы писали еще весной, для увеличения сбыта...  10312  positive
8259  Но молодой министр национальной экономики Биши...  10313  negative
8260  \n \nВ ЕНПФ назначен новый председатель правле...  10314   neutral
8261  В Алматы у отделения банка произошло нападение...  10315  negative
8262  НПП РК «Атамекен» предлагает создать Националь...  10316   neutral

[8263 rows x 3 columns]


In [ ]:
newtrain=pd.DataFrame(data=train,columns=['text','sentiment'])
print(newtrain)

                                                   text sentiment
0     Досудебное расследование по факту покупки ЕНПФ...  negative
1     Медики рассказали о состоянии пострадавшего му...  negative
2     Прошел почти год, как железнодорожным оператор...  negative
3     По итогам 12 месяцев 2016 года на территории р...  negative
4     Астана. 21 ноября. Kazakhstan Today - Агентств...  negative
...                                                 ...       ...
8258  Как мы писали еще весной, для увеличения сбыта...  positive
8259  Но молодой министр национальной экономики Биши...  negative
8260  \n \nВ ЕНПФ назначен новый председатель правле...   neutral
8261  В Алматы у отделения банка произошло нападение...  negative
8262  НПП РК «Атамекен» предлагает создать Националь...   neutral

[8263 rows x 2 columns]


In [ ]:
newtrain['sentiment'].where(~(newtrain.sentiment == 'negative'), other=int(0), inplace=True) 
newtrain['sentiment'].where(~(newtrain.sentiment == 'neutral'), other=int(0.5), inplace=True) 
newtrain['sentiment'].where(~(newtrain.sentiment == 'positive'), other=int(1), inplace=True) 

print(newtrain)


                                                   text sentiment
0     Досудебное расследование по факту покупки ЕНПФ...         0
1     Медики рассказали о состоянии пострадавшего му...         0
2     Прошел почти год, как железнодорожным оператор...         0
3     По итогам 12 месяцев 2016 года на территории р...         0
4     Астана. 21 ноября. Kazakhstan Today - Агентств...         0
...                                                 ...       ...
8258  Как мы писали еще весной, для увеличения сбыта...         1
8259  Но молодой министр национальной экономики Биши...         0
8260  \n \nВ ЕНПФ назначен новый председатель правле...         1
8261  В Алматы у отделения банка произошло нападение...         0
8262  НПП РК «Атамекен» предлагает создать Националь...         1

[8263 rows x 2 columns]


разделяем данные на учебу и ответы


In [ ]:
reviews = newtrain['text']

y_train = newtrain['sentiment'].astype('int64')


In [ ]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
8258    1
8259    0
8260    1
8261    0
8262    1
Name: sentiment, Length: 8263, dtype: int64

In [ ]:

reviews[:5]

0    Досудебное расследование по факту покупки ЕНПФ...
1    Медики рассказали о состоянии пострадавшего му...
2    Прошел почти год, как железнодорожным оператор...
3    По итогам 12 месяцев 2016 года на территории р...
4    Астана. 21 ноября. Kazakhstan Today - Агентств...
Name: text, dtype: object

In [ ]:
num_words = 10000
max_review_len = 1500

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(reviews)

In [ ]:
tokenizer.word_index

{'в': 1,
 'и': 2,
 'на': 3,
 'по': 4,
 'с': 5,
 'что': 6,
 'не': 7,
 'года': 8,
 'для': 9,
 'за': 10,
 'а': 11,
 'к': 12,
 'из': 13,
 '–': 14,
 'тенге': 15,
 '—': 16,
 'как': 17,
 'о': 18,
 'это': 19,
 'до': 20,
 'от': 21,
 '1': 22,
 'году': 23,
 'также': 24,
 '2016': 25,
 'ао': 26,
 'мы': 27,
 'казахстана': 28,
 'казахстан': 29,
 'то': 30,
 'рк': 31,
 '2': 32,
 'будет': 33,
 'при': 34,
 'но': 35,
 '5': 36,
 'области': 37,
 'все': 38,
 'млрд': 39,
 'его': 40,
 'республики': 41,
 'этом': 42,
 'он': 43,
 'у': 44,
 '3': 45,
 'более': 46,
 'развития': 47,
 'лет': 48,
 '4': 49,
 'их': 50,
 'или': 51,
 'так': 52,
 'г': 53,
 'было': 54,
 'уже': 55,
 '2017': 56,
 'том': 57,
 '2015': 58,
 '10': 59,
 'алматы': 60,
 'время': 61,
 'которые': 62,
 '6': 63,
 'был': 64,
 'только': 65,
 'год': 66,
 'есть': 67,
 'же': 68,
 'будут': 69,
 'казахстане': 70,
 '7': 71,
 'страны': 72,
 'млн': 73,
 'компании': 74,
 'я': 75,
 'тоо': 76,
 'kz': 77,
 'банка': 78,
 'со': 79,
 '8': 80,
 'экономики': 81,
 'были': 8

In [ ]:
sequences = tokenizer.texts_to_sequences(reviews)

In [ ]:
index = 0
print(reviews[index])
print(sequences[index])

Досудебное расследование по факту покупки ЕНПФ пакета облигаций ТОО "Бузгул Аурум" было начато по инициативе Национального банка РК, сообщил директор департамента защиты прав потребителей и финансовых услуг Нацбанка Казахстана Александр Терентьев.
"Основанием для досудебного расследования стало обращение Национального банка, письмо от 25 ноября 2016 года. Было обращение Национального банка в правоохранительные органы. Нам эта сделка показалась сомнительной, недостаточно корректной, поэтому Нацбанк 25 ноября 2016 года обратился в правоохранительные органы. Это то, что я могу озвучить на сегодня. Идёт следствие, проводится проверка", – сказал Терентьев.
28 декабря в Нацбанке заявили, что не знают, что стало основанием для проверки ЕНПФ.
23 декабря факт проведения проверки в АО "Единый накопительный пенсионный фонд" подтвердился. Пресс-служба Национального банка сообщила, что проверку проводят по операциям, совершённым АО "ЕНПФ" в отношении инвестирования собственных активов.
Также в финр

In [ ]:
x_train = pad_sequences(sequences, maxlen=max_review_len)

In [ ]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
8258    1
8259    0
8260    1
8261    0
8262    1
Name: sentiment, Length: 8263, dtype: int64

In [ ]:
x_train[0]

array([   0,    0,    0, ..., 1423, 1755, 1744], dtype=int32)

ПРОВЕРИТЬ ПАРАМЕТРЫ EMBEDDING
***Текст, выделенный полужирным шрифтом***


In [ ]:
model = Sequential()
model.add(Embedding(num_words, 64, input_length=max_review_len))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_save_path = 'drive/MyDrive/best_model.h5'
checkpoint_callback = ModelCheckpoint(model_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[checkpoint_callback])

Epoch 1/5
52/52 [==============================] - 68s 1s/step - loss: 0.4807 - accuracy: 0.8203 - val_loss: 0.4020 - val_accuracy: 0.8282

Epoch 00001: val_accuracy improved from -inf to 0.82819, saving model to drive/MyDrive/best_model.h5
Epoch 2/5
52/52 [==============================] - 7s 127ms/step - loss: 0.2939 - accuracy: 0.8752 - val_loss: 0.3162 - val_accuracy: 0.8651

Epoch 00002: val_accuracy improved from 0.82819 to 0.86509, saving model to drive/MyDrive/best_model.h5
Epoch 3/5
52/52 [==============================] - 7s 129ms/step - loss: 0.1847 - accuracy: 0.9238 - val_loss: 0.3477 - val_accuracy: 0.8609

Epoch 00003: val_accuracy did not improve from 0.86509
Epoch 4/5
52/52 [==============================] - 7s 126ms/step - loss: 0.1232 - accuracy: 0.9555 - val_loss: 0.3854 - val_accuracy: 0.8518

Epoch 00004: val_accuracy did not improve from 0.86509
Epoch 5/5
52/52 [==============================] - 7s 126ms/step - loss: 0.1010 - accuracy: 0.9635 - val_loss: 0.4045 -

In [ ]:
model.load_weights(model_save_path)

In [ ]:
test=pd.read_json('drive/MyDrive/test.json')  

In [ ]:
newtest=pd.DataFrame(data=test,columns=['text'])
newtest

,text
0,"Как сообщает пресс-служба акимата Алматы, для ..."
1,Казахстанские авиакомпании перевозят 250 тысяч...
2,На состоявшемся под председательством Касым-Жо...
3,В ОАЭ состоялись переговоры между казахстанско...
4,12 вагонов грузового поезда сошли с путей в Во...
...,...
2051,На официальной странице общественного движения...
2052,"официальный курс – 330,55 тенге за Доллар США ..."
2053,"«Базовая ставка, которая сейчас составляет 12%..."
2054,На начальном этапе за неоплату парковки на при...


In [ ]:
test_sequences = tokenizer.texts_to_sequences(newtest['text'])

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max_review_len)

In [ ]:
x_test[:1]

array([[  0,   0,   0, ...,  20, 160,   1]], dtype=int32)

In [ ]:
result = model.predict(x_test)

In [ ]:
index_test=2

In [ ]:
newtest['text'][index_test]

'На состоявшемся под председательством Касым-Жомарта Токаева заседании Сената депутаты верхней палаты обсудили отчеты правительства и Счетного комитета об исполнении республиканского бюджета за 2015 год .С основными докладами выступили руководители Министерства финансов, Счетного комитета по контролю за исполнением республиканского бюджета и Национального банка.Министр финансов Бахыт Султанов доложил, что в 2015 году доходы республиканского бюджета достигли 6 137 млрд тенге, или 105,7% к плану. Расходы составили 7 трлн 163 млрд тенге, что составляет 99,8% к плану.Приоритетом для правительства является социальная сфера. Контрцикличное увеличение бюджета, как подчеркнул Б.Султанов, в первую очередь произошло за счет роста социальных расходов, доля которых в структуре бюджета увеличилась до 39,4% и составила 2,8 трлн тенге. Размеры пенсионных выплат повышены на 9%.По данным Министерства финансов, в минувшем году отмечено снижение дефицита республиканского бюджета, который составил 912 млр

In [ ]:
result[index_test]

array([0.9423534], dtype=float32)

In [ ]:
import json

In [ ]:
rg = pd.read_json('drive/MyDrive/rg_all.json', encoding="utf-8-sig")

In [ ]:
newrg=pd.DataFrame(data=rg,columns=['Текст Статьи'])
newrg

,Текст Статьи
0,Депутат Госдумы от Республики Крым Руслан Баль...
1,Владимир Путин поздравил Никола Пашиняна с наз...
2,Google скоро запустит новый вариант подписки -...
3,В Южно-Африканской Республике после 14 лет стр...
4,В Костроме завершено расследование уголовного ...
...,...
295,Компания Searchlight Pictures опубликовала тре...
296,В Пятигорске мужчину заподозрили в порче сосед...
297,Российский гимнаст Давид Белявский остановился...
298,США вернули Ираку 17 тысяч археологических арт...


In [ ]:
rg_sequences = tokenizer.texts_to_sequences(newrg['Текст Статьи'])

In [ ]:
rg_test = pad_sequences(rg_sequences, maxlen=max_review_len)

In [ ]:
rg_test

array([[   0,    0,    0, ...,   18,   12,  180],
       [   0,    0,    0, ..., 3005, 9645,  546],
       [   0,    0,    0, ...,  867,   51, 1128],
       ...,
       [   0,    0,    0, ...,  252,  160, 1034],
       [   0,    0,    0, ..., 3704,    3, 7072],
       [   0,    0,    0, ...,    3, 2093,   91]], dtype=int32)

In [ ]:
result_rg = model.predict(rg_test)

In [ ]:
for i in range(0,106):
  print(newrg['Текст Статьи'][i])
  print(result_rg[i])


Депутат Госдумы от Республики Крым Руслан Бальбек назвал ахинеей заявление главы МИД Украины Дмитрия Кулебы, пригрозившего России "камнями с неба" из-за Крыма.- Кулебе нравится слушать самого себя и неважно, о чем идет речь - о реальных вещах или об ахинее, - сказал депутат RT. - Выражение "камни с неба" Кулеба использовал для красного словца. Главное - выступление с апломбом перед мировой аудиторией, все остальное вторично. Во всем идет европейский подход и даже воспоминания о том, как Франция свои провинции возвращала.Бальбек напомнил Кулебе о демократическом волеизъявлении крымчан при возвращении полуострова в РФ в 2014 году. И здесь совершенно неуместны исторические параллели с Францией, которая вернула себе регион Эльзас, потерянный в результате франко-прусской войны в 1871 году. В Крыму в 2014 году никакие военные действия не велись, а крымчане десятилетиями мечтали о присоединении к России.
[0.2519223]
Владимир Путин поздравил Никола Пашиняна с назначением на пост премьер-минист

In [ ]:
mtext = '''я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер я восхитительно делишз! классный супер прям отличный урок супер пупер'''

In [ ]:
msequence = tokenizer.texts_to_sequences([mtext])

In [ ]:
mdata = pad_sequences(msequence, maxlen=max_review_len)
mresult = model.predict(mdata)
mresult

array([[0.97877985]], dtype=float32)

# Новый раздел